# A/B testing: A step-by-step guide in Python

Для наших даних ми використовуватимемо <a href='https://www.kaggle.com/zhangluyuan/ab-testing?select=ab_data.csv'>набір даних з Kaggle</a>, який містить результати A/B-тесту двох різних дизайнів сторінки вебсайту (old_page і new_page). Ось що ми зробимо:

1. [Дизайн експерименту](#1.-Дизайн-експерименту)
2. [Збір і підготовка даних](#2.-Збір-і-підготовка-даних)
3. [Візуалізація результатів](#3.-Візуалізація-результатів)
4. [Тестування гіпотези](#4.-Тестування-гіпотези)
5. [Висновки](#5.-Висновки)

Щоб зробити це трохи реалістичнішим, ось потенційний **сценарій** для нашого дослідження:

> Уявіть, що ви працюєте в групі розробників **підприємства електронної комерції в Інтернеті**. Дизайнер UX дуже старанно попрацював над новою версією сторінки продукту, сподіваючись, що це призведе до більш високого коефіцієнта конверсії. Менеджер із продукту (PM) сказав вам, що **поточний коефіцієнт конверсії** становить у середньому **13%** протягом року, і що команда буде задоволена **збільшенням на 2%**, що означає, що новий дизайн вважатимуть успішним, якщо він підніме коефіцієнт конверсії до 15%.

Перед впровадженням зміни команді буде зручніше протестувати її на невеликій кількості користувачів, щоб побачити, як вона працює, тому ви пропонуєте виконати **A/B-тест** на підмножині користувачів з вашої користувацької бази.

***
## 1. Дизайн експерименту

### Формулювання гіпотези

Насамперед, ми хочемо переконатися, що сформулювали гіпотезу на початку нашого проєкту. Це забезпечить правильність і точність нашої інтерпретації результатів.

Оскільки ми не знаємо, чи буде новий дизайн працювати краще або гірше (або буде таким самим?), ми виберемо <a href="https://en.wikipedia.org/wiki/One-_and_two-tailed_tests">**двосторонній тест**</a>:

$$H_0: p = p_0$$
$$H_a: p \ne p_0$$

де $p$ і $p_0$ позначають коефіцієнт конверсії нового і старого дизайну відповідно. Ми також встановимо **рівень значущості 95%**

$$\alpha = 0.05$$

Значення $\alpha$ - це встановлений нами поріг, за яким ми говоримо: "Якщо ймовірність отримання результату як екстремального або більшого ($p$ - значення) є нижчою, ніж $\alpha$, то ми відхиляємо нульову гіпотезу". Оскільки наше $\alpha = 0.05$ (що вказує на ймовірність 5%), наша впевненість (1 - $\alpha$) становить 95%.

Якщо ви не знайомі з вищезазначеним, усе це насправді означає, що хоч би який коефіцієнт конверсії ми не спостерігали для нашого нового дизайну в нашому тесті, ми хочемо бути впевнені на 95%, що він статистично відрізняється від коефіцієнта конверсії нашого старого дизайну, перш ніж ми вирішимо відкинути нульову гіпотезу $H_0$.

### Вибір змінних

Для нашого тесту нам знадобляться **дві групи**:
* Група `control` - їм буде показано старий дизайн
* Група `treatment` (або експериментальна, тестова) група - їм буде показано новий дизайн.

Група - це буде наша *незалежна змінна* в цих даних (тобто Х). Причина, через яку в нас є дві групи, хоча ми знаємо базовий коефіцієнт конверсії, полягає в тому, що ми хочемо контролювати інші змінні, що можуть вплинути на наші результати, як-от сезонність: маючи `control` групу, ми можемо безпосередньо порівнювати їхні результати з групою `treatment`, тому що єдина систематична відмінність між групами - це дизайн сторінки продукту, і тому ми можемо пов'язати будь-які зміни в результатах із дизайном.

Для нашої *залежної змінної* (тобто того, що ми намагаємося виміряти, у) ми зацікавлені в реєстрації "коефіцієнта конверсії". Ми можемо закодувати це за допомогою кожного користувацького сеансу з двійковою змінною:
* `0` - користувач не купував товар під час цієї користувацької сесії.
* `1` - користувач купив продукт під час цієї користувацької сесії.

Таким чином, ми можемо легко обчислити середнє значення купівель для кожної групи, щоб отримати коефіцієнт конверсії кожного дизайну.

### Вибір розміру вибірки

Важливо зазначити, що, оскільки ми не тестуватимемо всю базу користувачів (нашу <a href = "https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/3-populations-and-samples"> популяцію </a>), отримані нами показники конверсії неминуче будуть лише *оцінками* справжніх коефіцієнтів.

Кількість людей (або користувацьких сеансів), яких ми вирішуємо охопити в кожній групі, вплине на точність наших розрахункових показників конверсії: **чим більший розмір вибірки**, тим точніші наші оцінки (тобто тим менші наші інтервали впевненості - confidence intervals), **тим вищий шанс виявити різницю** у двох групах, якщо вона є.

З іншого боку, чим більшою стає наша вибірка, тим дорожчим (і непрактичнішим) стає наше дослідження.

*Отже, скільки людей має бути в кожній групі?*

Розмір необхідної нам вибірки оцінюють за допомогою так званого  [*Аналізу потужності*](https://machinelearningmastery.com/statistical-power-and-power-analysis-in-python/), і це залежить від кількох факторів:
* **Потужність тесту** ($ 1 - \beta $) - являє собою ймовірність виявлення статистичної різниці між групами в нашому тесті, коли різниця справді присутня. Зазвичай це значення встановлюється на 0.8 (більше про [статистичну потужність](https://uk.wikipedia.org/wiki/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%BD%D0%B0_%D0%BF%D0%BE%D1%82%D1%83%D0%B6%D0%BD%D1%96%D1%81%D1%82%D1%8C) див. тут)
* **Альфа-значення** ($ \alpha $) - критичне значення, яке ми раніше встановили рівним 0.05.
* **Величина ефекту** - кількісна величина результату (різниці між показниками конферсії), наявного в дослідженні. Це статистичний показник, детальніше про який можна дізнатись [тут](https://machinelearningmastery.com/effect-size-measures-in-python/).

Оскільки наша команда була б задоволена різницею у 2%, ми можемо використати конверсії 13% і 15% для розрахунку очікуваного ефекту.

На щастя, **Python подбає про всі необхідні обчислення за нас**:

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

%matplotlib inline

In [ ]:
effect_size = sms.proportion_effectsize(0.15, 0.13)    # Розрахунок розміру ефекту на основі наших очікуваних показників

In [ ]:
effect_size

In [ ]:
required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1
    )                                                  # Розрахунок необхідного розміру вибірки

required_n = ceil(required_n)                          # Округлення до наступного цілого числа

print(required_n)

Нам буде потрібно **мінімум 4720 спостережень для кожної групи**.

Встановлення параметра power на 0.8 на практиці означає, що якщо існує реальна різниця в коефіцієнті конверсії між нашими версіями продукту, припускаючи, що різниця є тією, яку ми оцінили (13% проти 15%),
то ми маємо 80% шанс визначити його як статистично значущий у нашому тесті з розміром вибірки, який ми розрахували.

Розмір вибірки також можна обчислити на сайтах, загугливши "sample size ab test calculator", наприклад - тут https://www.evanmiller.org/ab-testing/sample-size.html, або тут https://www.abtasty.com/sample-size-calculator/.

***
## 2. Збір і підготовка даних

Отже, тепер, коли у нас є необхідний розмір вибірки, нам потрібно зібрати дані. Зазвичай на цьому етапі ви працюєте зі своєю командою над налаштуванням експерименту, імовірно, за допомогою групи інженерів.  Переконайтеся, що ви зібрали достатньо даних з точки зору необхідного розміру вибірки.

Наприклад,
однак, оскільки ми будемо використовувати набір відкритих даних, то ми:
1. Завантажимо <a href='https://www.kaggle.com/zhangluyuan/ab-testing?select=ab_data.csv'> набір даних з Kaggle </a>.
2. Зчитуємо дані у фрейм даних pandas.
3. Перевіримо і за необхідності очистимо дані.
4. Довільно виберемо n = 4720 рядків із DataFrame для кожної групи.

**Примітка**: Зазвичай нам не потрібно виконувати крок 4, це просто заради вправи.

In [ ]:
df = pd.read_csv('../data/ab_data/ab_data.csv.zip')

df.head()

In [ ]:
df.info()

In [ ]:
# Щоб переконатися, що вся контрольна група бачить стару сторінку і навпаки.

pd.crosstab(df['group'], df['landing_page'])

У DataFrame **294478 рядків**, кожен з яких представляє сеанс користувача, а також **5 стовпців**:
* `user_id` - ID користувача кожної сесії
* `timestamp` - позначка часу сеансу
* `group` - у яку групу було призначено користувача для цього сеансу {`control`, `treatment`}
* `landing_page` - який дизайн кожен користувач бачив у цьому сеансі {`old_page`, `new_page`}
* `convert` - чи завершився сеанс конверсією (двійкова змінна, `0` - немає конверсії, `1` - є конверсія)

Фактично ми використовуватимемо для аналізу тільки стовпці "group" і "convert".

Перш ніж ми продовжимо і проаналізуємо дані для отримання нашої підмножини, давайте упевнимося, що немає користувачів, які були обрані кілька разів.

In [ ]:
session_counts = df['user_id'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'Є {multi_users} користувачів, які зустрічаються кілька разів у наборі даних.')

Насправді є користувачі, які з'являються більше одного разу. Оскільки це число порівняно мале, ми видалимо їх із DataFrame, щоб уникнути вибірки одних і тих самих користувачів двічі.

In [ ]:
users_to_drop = session_counts[session_counts > 1].index

df = df[~df['user_id'].isin(users_to_drop)]
print(f'Новий розмір набору даних: {df.shape}')

### Самплінг

Тепер, коли наш DataFrame красивий і чистий, ми можемо продовжити і вибрати n=4720 записів для кожної з груп.

In [ ]:
control_sample = df[df['group'] == 'control'].sample(n=required_n, random_state=22)
treatment_sample = df[df['group'] == 'treatment'].sample(n=required_n, random_state=22)

ab_test = pd.concat([control_sample, treatment_sample], axis=0)
ab_test.reset_index(drop=True, inplace=True)

In [ ]:
ab_test

In [ ]:
ab_test.info()

In [ ]:
ab_test['group'].value_counts()

Тепер ми готові аналізувати наші результати.

***
## 3. Візуалізація результатів

Перше, що ми можемо зробити, це обчислити деяку **базову статистику**, щоб отримати уявлення про те, який вигляд мають наші зразки.

In [ ]:
conversion_rates = ab_test.groupby('group')['converted']

std_p = lambda x: np.std(x)              # Std. deviation
se_p = lambda x: stats.sem(x)            # (std / sqrt(n)) - standard error of the mean.

conversion_rates = conversion_rates.agg([np.mean, std_p, se_p])
conversion_rates.columns = ['conversion_rate', 'std_deviation', 'std_error']


conversion_rates.style.format('{:.3f}')

Судячи зі статистики вище, **наші два дизайни працювали дуже схоже**, але наш новий дизайн працював трохи краще, прибл. **коефіцієнт конверсії 12,3% проти 12,6%**.

Нанесення даних на графік спростить розуміння цих результатів:

In [ ]:
plt.figure(figsize=(8,6))

sns.barplot(x=ab_test['group'], y=ab_test['converted'], ci=False)

plt.ylim(0, 0.17)
plt.title('Conversion rate by group', pad=20)
plt.xlabel('Group', labelpad=15)
plt.ylabel('Converted (proportion)', labelpad=15);

Коефіцієнти конверсії для наших груп дійсно дуже близькі. Також зверніть увагу, що коефіцієнт конверсії контрольної групи нижчий, ніж те, що ми очікували, виходячи зі знань про наш середній коефіцієнт конверсії (12,3% проти 13%). Це свідчить про те, що є деякі відмінності в результатах під час добору вибірки із сукупності.

Отже, значення групи `treatment` вище. **Чи є ця різниця *статистично значущою***?

***
## 4. Тестування гіпотези

Останній крок нашого аналізу - перевірка нашої гіпотези. Оскільки у нас дуже велика вибірка, ми можемо використовувати <a href="https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval#Normal_approximation_interval"> нормальне наближення </a> для обчислення нашого значення $p$ (тобто z-тест).

Знову ж таки, Python спрощує всі обчислення. Ми можемо використовувати модуль statsmodels.stats.proportion, щоб отримати значення $p$ і довірчі інтервали:

In [ ]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [ ]:
control_results = ab_test[ab_test['group'] == 'control']['converted']
treatment_results = ab_test[ab_test['group'] == 'treatment']['converted']

In [ ]:
n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'Довірчий інтервал 95% для групи control: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Довірчий інтервал 95% для групи treatment: [{lower_treat:.3f}, {upper_treat:.3f}]')


***
## 5. Висновки

Оскільки наше $p$-значення=0.732 набагато вище за наше $\alpha$ = 0.05, ми не можемо відхилити нульову гіпотезу $H_0$, що означає, що наш новий дизайн не працював суттєво (не кажучи вже що краще), ніж наш старий.

Крім того, якщо ми подивимося довірчий інтервал для групи `treatment` ([0,116, 0,135], тобто 11,6-13,5%), ми помітимо, що:
1. він включає наше базове значення коефіцієнта конверсії 13%.
2. Він не включає наше цільове значення в 15% (зростання на 2%, до якого ми прагнули).

Це означає, що більш імовірно, що істинний коефіцієнт конверсії нового дизайну буде схожий на наш базовий рівень, а не на цільові 15%, на які ми сподівалися. Це ще один доказ того, що наш новий дизайн навряд чи стане поліпшенням нашого старого. Тож потрібно придумати що-небудь інше :)